In [1]:
# Imports
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from imageio import imread
import warnings
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from IPython.display import display
import seaborn as sns
from wordcloud import WordCloud, ImageColorGenerator

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict


import altair as alt
from time import time

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/renatolopo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# refinando stop word
stop_words = set(stopwords.words("portuguese"))
stop_words.update(['que', 'até', 'esse', 
                    'essa', 'pro', 'pra',
                    'oi', 'lá', 'blá', 'dos', 'esta'])
const_words = [ 'de','a','o','que','e','do','da','em','um','para','e','com','nao','uma','os','no','vou','se','na','por','mais','as','dos','como','mas','foi','ao','ele','das','tem','a','seu','sua','ou','ser','quando','muito','ha','nos','ja','esta','eu','tambem','so','pelo','pela','ate','isso','ela','entre','era','depois','sem','mesmo','aos','ter','seus','quem','nas','me','esse','eles','estao','voce','tinha','foram','essa','num','nem','suas','meu','as','minha','tem','numa','pelos','elas','havia','seja','qual','sera','nos','tenho','lhe','deles','essas','esses','pelas','este','fosse','dele','tu','te','voces','vos','lhes','meus','minhas','teu','tua','teus','tuas','nosso','nossa','nossos','nossas','dela','delas','esta','estes','estas','aquele','aquela','aqueles','aquelas','isto','aquilo','estou','esta','estamos','estao','estive','esteve','estivemos','estiveram','estava','estavamos','estavam','estivera','estiveramos','esteja','estejamos','estejam','estivesse','estivessemos','estivessem','estiver','estivermos','estiverem','hei','ha','havemos','hao','houve','houvemos','houveram','houvera','houveramos','haja','hajamos','hajam','houvesse','houvessemos','houvessem','houver','houvermos','houverem','houverei','houvera','houveremos','houverao','houveria','houveriamos','houveriam','sou','somos','sao','era','eramos','eram','fui','foi','fomos','foram','fora','foramos','seja','sejamos','sejam','fosse','fossemos','fossem','for','formos','forem','serei','sera','seremos','serao','seria','seriamos','seriam','tenho','tem','temos','tem','tinha','tinhamos','tinham','tive','teve','tivemos','tiveram','tivera','tiveramos','tenha','tenhamos','tenham','tivesse','tivessemos','tivessem','tiver','tivermos','tiverem','terei','tera','teremos','terao','teria','teriamos','teriam','a', 'à', 'adeus', 'agora', 'aí', 'ainda', 'além', 'algo', 'alguém', 'algum', 'alguma', 'algumas', 'alguns', 'ali', 'ampla', 'amplas', 'amplo', 'amplos', 'ano', 'anos', 'ante', 'antes', 'ao', 'aos', 'apenas', 'apoio', 'após', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aqui', 'aquilo', 'área', 'as', 'às', 'assim', 'até', 'atrás', 'através', 'baixo', 'bastante', 'bem', 'boa', 'boas', 'bom', 'bons', 'breve', 'cá', 'cada', 'catorze', 'cedo', 'cento', 'certamente', 'certeza', 'cima', 'cinco', 'coisa', 'coisas', 'com', 'como', 'conselho', 'contra', 'contudo', 'custa', 'da', 'dá', 'dão', 'daquela', 'daquelas', 'daquele', 'daqueles', 'dar', 'das', 'de', 'debaixo', 'dela', 'delas', 'dele', 'deles', 'demais', 'dentro', 'depois', 'desde', 'dessa', 'dessas', 'desse', 'desses', 'desta', 'destas', 'deste', 'destes', 'deve', 'devem', 'devendo', 'dever', 'deverá', 'deverão', 'deveria', 'deveriam', 'devia', 'deviam', 'dez', 'dezanove', 'dezasseis', 'dezassete', 'dezoito', 'dia', 'diante', 'disse', 'disso', 'disto', 'dito', 'diz', 'dizem', 'dizer', 'do', 'dois', 'dos', 'doze', 'duas', 'dúvida', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'embora', 'enquanto', 'entre', 'era', 'eram', 'éramos', 'és', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estás', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estiveste', 'estivestes', 'estou', 'etc', 'eu', 'exemplo', 'faço', 'falta', 'favor', 'faz', 'fazeis', 'fazem', 'fazemos', 'fazendo', 'fazer', 'fazes', 'feita', 'feitas', 'feito', 'feitos', 'fez', 'fim', 'final', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'forma', 'formos', 'fosse', 'fossem', 'fôssemos', 'foste', 'fostes', 'fui', 'geral', 'grande', 'grandes', 'grupo', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'havia', 'hei', 'hoje', 'hora', 'horas', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvéssemos', 'isso', 'isto', 'já', 'la', 'lá', 'lado', 'lhe', 'lhes', 'lo', 'local', 'logo', 'longe', 'lugar', 'maior', 'maioria', 'mais', 'mal', 'mas', 'máximo', 'me', 'meio', 'menor', 'menos', 'mês', 'meses', 'mesma', 'mesmas', 'mesmo', 'mesmos', 'meu', 'meus', 'mil', 'minha', 'minhas', 'momento', 'muita', 'muitas', 'muito', 'muitos', 'na', 'nada', 'não', 'naquela', 'naquelas', 'naquele', 'naqueles', 'nas', 'nem', 'nenhum', 'nenhuma', 'nessa', 'nessas', 'nesse', 'nesses', 'nesta', 'nestas', 'neste', 'nestes', 'ninguém', 'nível', 'no', 'noite', 'nome', 'nos', 'nós', 'nossa', 'nossas', 'nosso', 'nossos', 'nova', 'novas', 'nove', 'novo', 'novos', 'num', 'numa', 'número', 'nunca', 'o', 'obra', 'obrigada', 'obrigado', 'oitava', 'oitavo', 'oito', 'onde', 'ontem', 'onze', 'os', 'ou', 'outra', 'outras', 'outro', 'outros', 'para', 'parece', 'parte', 'partir', 'paucas', 'pela', 'pelas', 'pelo', 'pelos', 'pequena', 'pequenas', 'pequeno', 'pequenos', 'per', 'perante', 'perto', 'pode', 'pude', 'pôde', 'podem', 'podendo', 'poder', 'poderia', 'poderiam', 'podia', 'podiam', 'põe', 'põem', 'pois', 'ponto', 'pontos', 'por', 'porém', 'porque', 'porquê', 'posição', 'possível', 'possivelmente', 'posso', 'pouca', 'poucas', 'pouco', 'poucos', 'primeira', 'primeiras', 'primeiro', 'primeiros', 'própria', 'próprias', 'próprio', 'próprios', 'próxima', 'próximas', 'próximo', 'próximos', 'pude', 'puderam', 'quais', 'quáis', 'qual', 'quando', 'quanto', 'quantos', 'quarta', 'quarto', 'quatro', 'que', 'quê', 'quem', 'quer', 'quereis', 'querem', 'queremas', 'queres', 'quero', 'questão', 'quinta', 'quinto', 'quinze', 'relação', 'sabe', 'sabem', 'são', 'se', 'segunda', 'segundo', 'sei', 'seis', 'seja', 'sejam', 'sejamos', 'sem', 'sempre', 'sendo', 'ser', 'será', 'serão', 'serei', 'seremos', 'seria', 'seriam', 'seríamos', 'sete', 'sétima', 'sétimo', 'seu', 'seus', 'sexta', 'sexto', 'si', 'sido', 'sim', 'sistema', 'só', 'sob', 'sobre', 'sois', 'somos', 'sou', 'sua', 'suas', 'tal', 'talvez', 'também', 'tampouco', 'tanta', 'tantas', 'tanto', 'tão', 'tarde', 'te', 'tem', 'tém', 'têm', 'temos', 'tendes', 'tendo', 'tenha', 'tenham', 'tenhamos', 'tenho', 'tens', 'ter', 'terá', 'terão', 'terceira', 'terceiro', 'terei', 'teremos', 'teria', 'teriam', 'teríamos', 'teu', 'teus', 'teve', 'ti', 'tido', 'tinha', 'tinham', 'tínhamos', 'tive', 'tivemos', 'tiver', 'tivera', 'tiveram', 'tivéramos', 'tiverem', 'tivermos', 'tivesse', 'tivessem', 'tivéssemos', 'tiveste', 'tivestes', 'toda', 'todas', 'todavia', 'todo', 'todos', 'trabalho', 'três', 'treze', 'tu', 'tua', 'tuas', 'tudo', 'última', 'últimas', 'último', 'últimos', 'um', 'uma', 'umas', 'uns', 'vai', 'vais', 'vão', 'vários', 'vem', 'vêm', 'vendo', 'vens', 'ver', 'vez', 'vezes', 'viagem', 'vindo', 'vinte', 'vir', 'você', 'vocês', 'vos', 'vós', 'vossa', 'vossas', 'vosso', 'vossos', 'zero', 'the', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '_' ]
wrd = [w for w in const_words if w not in stop_words]
# contem 593 stop words em portugues
stop_words.update(wrd)

In [3]:
#import data
file_name = "Twitter"
df = pd.read_csv(f'/home/renatolopo/notebooks/files/resultados final/{file_name}.csv',  encoding='utf-8',  lineterminator='\n')

In [4]:
# remove special characters and digits
def clean_noticia(noticia):
   
    # remove links e alguns pontos
    noticia = re.sub(r'<img src="https\S+', "", noticia)
    noticia = re.sub(r'href\S+', "", noticia)
    noticia = re.sub(r'rel', "", noticia)
    noticia = re.sub(r'nofollow\S+', "", noticia)
    noticia = re.sub(r'mixvale\S+', "", noticia)
    noticia = re.sub(r'target\S+', "", noticia)
    noticia = re.sub(r'_blank\S+', "", noticia)
    noticia = re.sub(r'www\S+', "", noticia)
    noticia = re.sub(r"http\S+", "", noticia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    
    # remove alguns caracteres
    noticia  = re.sub("(\\d|\\W)+|\w*\d\w*"," ",noticia )
    noticia = ' '.join(s for s in noticia.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
    noticia = noticia.replace("\n", "")
    
    #remove stopwords
    noticia = ' '.join([w for w in noticia.split() if w not in stop_words])
    return noticia

def clean_tweet(tweet):
    # remove os RT
    tweet = re.sub(r'RT+', '', tweet) 
    
    # remove as menções
    tweet = re.sub(r'@\S+', '', tweet)  
    
    # remove links e alguns pontos
    tweet = re.sub(r'kkk', '', tweet)
    tweet = re.sub(r'kkk\S+', '', tweet)
    tweet = re.sub(r"http\S+", "", tweet).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    
    # remove alguns caracteres
    tweet  = re.sub("(\\d|\\W)+|\w*\d\w*"," ",tweet )
    tweet = ' '.join(s for s in tweet.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
    tweet = tweet.replace("\n", "")
    tweet = ' '.join([w for w in tweet.split() if w not in stop_words])
    return tweet

## Noticia

In [5]:
clean_text = []
for w in range(len(df.SUMMARY)):
  # seleciona o titulo caso não exista descrição  
  if df['SUMMARY'].iloc[w] == 'nan' or df['SUMMARY'].iloc[w] == '-' or type(df['SUMMARY'].iloc[w]) !=" <class 'str'>":
    noticia = df['TITLE'].iloc[w]
  else:
    noticia = df['SUMMARY'].iloc[w]
 
  noticia = clean_noticia(str(noticia))
  
  clean_text.append(noticia)


print(len(clean_text))
#clean_text[:10]

156801


## Twitter

In [5]:
clean_text = []
for w in range(len(df.TWEET)):
  tweet = df['TWEET'].iloc[w]
  tweet = clean_tweet(tweet)
    
  clean_text.append(tweet)


#clean_tweets[110:145]

## Modelagem de Tópicos LDA

In [6]:
df['text'] = np.array(clean_text)

In [7]:
# COUNT vectorizer
tf_vectorizer = CountVectorizer(
        min_df = 30,
        max_df = 0.5,
        max_features = 10000,
        stop_words = stop_words, 
        ngram_range = (1,2)
  )

#transform
vec_text = tf_vectorizer.fit_transform(clean_text)

#returns a list of words.
words = tf_vectorizer.get_feature_names()
#print(clean_tweets)

print(len(words))

10000


In [8]:
from sklearn.decomposition import LatentDirichletAllocation

In [9]:

def get_topics(W, H, feature_names, documents, no_top_words, no_top_documents):
    topicos = {}
    for topic_idx, topic in enumerate(H):
        topicos[f'{topic_idx+1}'] = ", ".join([feature_names[i]
                for i in topic.argsort()[:-no_top_words - 1:-1]])
    return topicos

        
def display_topics(W, H, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("\n--\nTopic #{}: ".format(topic_idx + 1))
        print(", ".join([feature_names[i]
                for i in topic.argsort()[:-no_top_words - 1:-1]]).upper())
        top_d_idx = np.argsort(W[:,topic_idx])[::-1][0:no_top_documents]
        for d in top_d_idx: 
          doc_data = df[['TITLE', 'SUMMARY']].iloc[d]
          print(f'{doc_data[0]} -  \t{W[d, topic_idx]:.2f}')

In [ ]:
lda = LatentDirichletAllocation(n_components=15, 
                                learning_method='online', # 'online' equivale a minibatch no k-means
                                random_state=0)

t0 = time()

lda.fit(vec_text)
doc_topic_matrix = lda.transform(vec_text)

print("done in %0.3fs." % (time() - t0))

In [11]:
topicos = get_topics(doc_topic_matrix,
               lda.components_, 
               words,
               df,
               15, 
               10)

In [12]:
for i, v in topicos.items():
    print(f'{i} - {v}\n')

1 - governo, homem, prefeito, preso, candidatos, pix, eleição, ônibus, pensão, caso, empresas, recorde, homem preso, presos, concessão

2 - paulo, mulher, carro, entenda, fica, acidente, zona, motorista, cnh, centro, fogo, juiz, mata, bate, covas

3 - bolsonaro, morre, casa, saque, rio, janeiro, homem, jovem, conta, quase, causa, pesquisa, vítima, conheça, caminhão

4 - covid, vacinação, país, posse, poderá, corpo, renda, crianças, criança, profissionais, social, educação, comércio, maternidade, bbb

5 - fgts, caixa, anuncia, morto, cadastro, lança, aplicativo, tiros, brasileiro, moradores, cidades, cadastro único, mantém, vence, domingo

6 - vacina, prefeitura, durante, novembro, vídeo, veja, campanha, direito, regras, hospital, mostra, dicas, uso, estados, empresa

7 - volta, outubro, dezembro, vivo, santos, ministério, joão, cai, filho, público, governo, aulas, trabalhador, valores, pessoa

8 - vídeos, cidade, região, quintafeira, sextafeira, mercado, edição, dados, porto, preto, jo

In [13]:
# Função que pega uma lista de palavras e um texto como entrada e conta quantas vezes as palavras da lista aparecem no texto
def count_words(words, text):
    counter = 0
    for word in words:
        for w in text.split():
            if word == w:
                counter+= 1
    return counter

# Função para classificar cada resumo em um tópico, verificando o tópico que contém mais palavras.
# Para cada resumo, cria um dicionário {topic: qtd words da lista} e depois verifica qual é a chave no dicionário com o maior número, que será o número do tópico a ser adicionado a uma lista e devolvido.

def assign_topic(topic, text):
    assign = []
    for row in text:
        topic_counts = {}
        for key, value in topic.items():
            v = [x for x in value.split(',')]
            topic_counts[key] = count_words(v, row) #use function previously created
            
        ma = max(topic_counts, key=lambda k: topic_counts[k])
        if topic_counts[ma] == 0:
            assign.append('null')
        else:
             assign.append(ma)
       
    return assign

In [14]:
assign_topic(topicos, df['text'].sample(25))

['null',
 'null',
 '12',
 'null',
 'null',
 '10',
 'null',
 'null',
 'null',
 '15',
 'null',
 'null',
 'null',
 'null',
 '11',
 'null',
 '12',
 'null',
 '15',
 '15',
 'null',
 'null',
 'null',
 '12',
 'null']

In [17]:
# A lista de tópicos que irão no dataframe
topics_list = assign_topic(topicos, df.text)

# Adicionando a coluna de tópicos ao dataframe
df['topics'] = topics_list
#df.sample(25)

In [16]:
df['topics'].value_counts()

null    111692
15        7695
12        4318
8         4000
13        3882
11        3579
1         3556
10        3363
6         2321
5         2309
3         2173
14        2087
4         1848
2         1623
7         1289
9         1066
Name: topics, dtype: int64

In [18]:
def get_termos(topic):
    return topicos[str(topic)]

topicos['null'] = 'null'
df['termos'] = np.array([get_termos(topic) for topic in df['topics'].values.astype('U')])
#df.sample(25)

## Analise de sentimento

In [19]:
#dados de treino disponivel em https://www.kaggle.com/luisfredgs/imdb-ptbr
dataset = pd.read_csv('./files/treino/imdb-reviews-pt-br.csv')
df2 = pd.read_csv('./files/treino/Tweets_Mg.csv')
# unir os dois df
rm_df2 = [x for x in list(df2.columns) if x not in ['Text','Classificacao']]

#deixa os dois df com o mesmo numero de colunas
dataset = dataset.drop(columns=['text_en','id'])
df2 = df2.drop(columns=rm_df2)

#deixa as colunas de sentimentos igual a do outro df
dataset['sentiment'] = dataset['sentiment'].str.replace('neg', 'Negativo')
dataset['sentiment'] = dataset['sentiment'].str.replace('pos', 'Positivo')

# concatena os dois df
dataset.columns = ['Text', 'Polaridade']
df2.columns = ['Text', 'Polaridade']
dataset = pd.concat([dataset,df2])
#dataset

In [20]:
#modelo
text = dataset["Text"].values
classes = dataset["Polaridade"].values

vectorizer = CountVectorizer(analyzer = "word")
freq_text = vectorizer.fit_transform(text)

modelo = MultinomialNB()
modelo.fit(freq_text, classes)

MultinomialNB()

In [ ]:
# Validação cruzada do modelo. Neste caso, o modelo é dividido em 10 partes, treinado em 9 e testado em 1
resultados = cross_val_predict(modelo, freq_text, classes, cv = 10)
# acuracia
metrics.accuracy_score(classes, resultados)

### Classificando os textos

In [21]:
# Modelo treinado
# dado um texto, retorna sua polaridade
def classifica(text):
    text = [text]
    freq_testes = vectorizer.transform(text)
    return modelo.predict(freq_testes)

In [22]:
df['polaridade'] = np.array([classifica(text)[0] for text in df['TITLE'].values.astype('U')])

In [24]:
df.sample(25)

,TITLE,SUMMARY,text,topics,termos,polaridade
83595,Polícia Civil prende grupo de traficantes com ...,"Por meio de informações anônimas, o Grupo de D...",polícia civil prende traficantes quilos maconh...,13,"polícia, aposentadoria, suspeito, operação, pr...",Positivo
63950,Suposto cabrito maltratado é encontrado pela p...,A Polícia Civil foi chamada para atender uma s...,suposto cabrito maltratado encontrado polícia ...,13,"polícia, aposentadoria, suspeito, operação, pr...",Negativo
91722,Atingidos em Mariana querem CPI para investiga...,Entidade financiada por Vale e BHP é a respons...,atingidos mariana cpi investigar fundação renova,null,null,Positivo
54745,Teste do bafômetro de motorista que atropelou ...,Motorista foi autuado por homicídio culposo. E...,teste bafômetro motorista atropelou matou mili...,null,null,Positivo
37057,Homem é preso após marcar encontro com menina ...,Polícia chegou ao local marcado pelo pedófilo ...,homem preso marcar encontro menina pais garota...,13,"polícia, aposentadoria, suspeito, operação, pr...",Positivo
82169,Delegado envolvido em esquema ilegal em aeropo...,Policial colaborava para liberação de mercador...,delegado envolvido esquema ilegal aeroporto pe...,null,null,Positivo
33565,Camilo Santana anuncia tablets para os estudan...,"<img src=""https://s2.glbimg.com/dmjbYFne_LYYcW...",camilo santana anuncia tablets estudantes univ...,null,null,Positivo
77173,EUA rejeitam proposta de Putin para renovar pa...,Governo norte-americano rejeita ideia do presi...,eua rejeitam proposta putin renovar pacto nuclear,null,null,Positivo
56582,Matrículas para cursos de bordado e panificaçã...,"<img src=""https://s2.glbimg.com/ccFRoUNsT2DGTs...",matrículas cursos bordado panificação abertas ...,null,null,Neutro
76410,BRDE atinge marco de mais de R$ 945 milhões em...,O Banco Regional de Desenvolvimento do Extremo...,brde atinge marco milhões contratos paraná,null,null,Positivo


In [23]:
df.to_csv(f'/home/renatolopo/notebooks/files/resultados final/resultado_{file_name}.csv', index=False)